### ODA112 - Audit BackUp Response - PwC Connect

First, we import all the libraries we're going to work with across the application.

In [1]:
from boxsdk import OAuth2, Client
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import pandas

Then we log into our personal BOX account, through the ```Client ID```, ```Client Secret``` and ```Developer Token```. All of this variables are picked up from the application that was created in the BOX Developer Console.

In [14]:
authentication_method = OAuth2(client_id = '0voy64ajgdt0rxxibmeib0elopx7pufu', 
                               client_secret = 'G7UOuMABnMq4WYcjd9co88p7sAZFDMXU', 
                               access_token = 'bu8myZLWmX9NxMDhuIDNHhgEWBpCRWcd')
my_client = Client(authentication_method)

Then we obtain the Authentication URL for the BOX folder we've created in our personal BOX repository.

In [9]:
box_folder_link = 'https://app.box.com/folder/142802142764'
authentication_url, csrf_token = authentication_method.get_authorization_url(box_folder_link)
print(authentication_url)

https://account.box.com/api/oauth2/authorize?state=box_csrf_token_h5WpEjdol1CNoxAf&response_type=code&client_id=8f480o1hikhaclrk2xinr5oblchierem&redirect_uri=https%3A%2F%2Fapp.box.com%2Ffolder%2F142802142764


Interacting with Selenium, we open a new browser session - in this case with Google Chrome application - and refer to the Authentication URL. Through this course of action, we can automatically obtain the code that will be needed to retrieve the ```Access Token``` in the following step.  

In [ ]:
chrome = webdriver.Chrome(r"C:\Users\BrunoSecchiari\Desktop\ODA112 - Audit Backup Response - PwC Connect\Chrome Drive\chromedriver.exe")

chrome.get(authentication_url)

login = chrome.find_element_by_name("login")
login.send_keys("brunosecchiari95@gmail.com")
password = chrome.find_element_by_name("password")
password.send_keys("vwjpnz95")
login_submit_button = chrome.find_element_by_name("login_submit")
login_submit_button.click()
consent_accept_button = chrome.find_element_by_name("consent_accept")
consent_accept_button.click()

login = chrome.find_element_by_name("login")
login.send_keys("brunosecchiari95@gmail.com")
login_submit_button = chrome.find_element_by_id("login-submit")
login_submit_button.click()
password = chrome.find_element_by_name("password")
password.send_keys("vwjpnz95")
password_submit_button = chrome.find_element_by_id("login-submit-password")
password_submit_button.click()

my_string = chrome.current_url
position1 = re.search('code=', my_string)
starting_position = position1.span()
position2 = re.search('&state=', my_string)
ending_position = position2.span()
my_finalized_string = my_string[starting_position[1]:ending_position[0]]
my_finalized_string

We exchange the code that was part of the Authentication URL for the ```Access Token```, meaning we're now able to interact with BOX API.

In [10]:
authentication_code = 'pYzQ7kCBMnStsz7W56shkWG3IGjSaLnb'
access_token, refresh_token = authentication_method.authenticate(authentication_code)

Finally, we'll create all the functions that we're going to work with along the application.

##### Retrieve File ID Function

In [11]:
def get_file_id(client, file_name, folder_id):
    
    file_id = None
    file_name_with_extension = file_name.split('.')
    file_name = file_name_with_extension[0]
    file_extension = file_name_with_extension[1]
    
    my_list = my_client.search().query(query = f'"{file_name}"', extension = [f'{file_extension}'], limit = 1)
    
    for i in my_list:
        file_id = i.id
    if not file_id:
        raise Exception(f'File was not found in Folder ID: {folder_id}.')
    return file_id

In [12]:
my_file_id = get_file_id(my_client, 'List of Entity - Batch 1.xlsx', '142803598097')
my_file_id

'843349623986'

##### Create Shared Link Function

In [14]:
def create_shared_link(client, file_id):

    try:
        my_shared_link = None
        my_shared_link = client.file(file_id).get_shared_link(access = 'open', allow_download = True)
        print(f"Shared Link was successfully created for File ID '{file_id}'.")
        return my_shared_link
    
    except BoxAPIException as Error:
        raise Exception(f"An error ocurred while trying to create the Shared Link for File ID: '{file_id}'.")

In [15]:
create_shared_link(my_client, my_file_id)

Shared Link was successfully created for File ID '841995560519'.


'https://app.box.com/s/9ttk4g3mkonnqs3lnq35e74af2n9ha0i'

##### Retrieve Shared Link Function

In [16]:
def get_shared_link(client, file_id):
    
    try:
        my_shared_link = None
        my_shared_link = client.file(file_id).get().shared_link
        print(f"The Shared Link for File ID '{file_id}' is the following: ")
        return my_shared_link['download_url']
    
    except BoxAPIException as Error:
        raise Exception(f"An error ocurred while trying to obtain the Shared Link for File ID '{file_id}'.")

In [17]:
get_shared_link(my_client, my_file_id)

The Shared Link for File ID '841995560519' is the following: 


'https://app.box.com/shared/static/9ttk4g3mkonnqs3lnq35e74af2n9ha0i.xlsx'

##### Upload Function

In [18]:
def upload_file(client, folder_id, file_path):
    
    try:
        file_for_upload = None
        file_for_upload = client.folder(folder_id).upload(file_path)
        print(f"File was successfully uploaded to BOX Folder '{folder_id}' with the following File ID '{file_for_upload.id}'.")
    
    except BoxAPIException as Error:
        raise Exception(f"An error ocurred while trying to upload the file to BOX Folder '{folder_id}'.")

In [26]:
upload_file(my_client, '140701906674', 
            r"C:\Users\BrunoSecchiari\Desktop\ODA112 - Audit Backup Response - PwC Connect\List of Entity - Batch 1.xlsx")

File was successfully uploaded to BOX Folder '140701906674' with the following File ID '841995560519'.


##### Download Function

In [19]:
def download_file(client, file_id, file_path):
    
    try:
        download_path = None
        download_path = open(file_path, 'wb')
        client.file(file_id).download_to(download_path)
        print(f"File ID '{file_id}' was successfully downloaded.")
        
    except BoxAPIException as Error:
        raise Exception(f"An error ocurred while trying to download File ID '{file_id}'.")

In [20]:
download_file(my_client, my_file_id, r"C:\Users\BrunoSecchiari\Desktop\List of Entity - Batch 1.xlsx")

File ID '841995560519' was successfully downloaded.
